In [1]:
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import re

In [2]:
html = """

"""

In [3]:
# Parse
soup = BeautifulSoup(html, "html.parser")

# Detect round (if present)
round_name = soup.select_one(".draw-header")
round_label = round_name.text.strip() if round_name else "Round"

# Find all matches
matches = soup.select(".draw-item")

# Initialize bracket with proper format
# 1. 🟢 ADAPTATION: Change to 'Bracket/32' for a 32-player draw
wiki_output = [f"{{{{Bracket|Bracket/32|id=XXXXXXXXXX"]

# 2. 🟢 ADAPTATION: Define matches per round for a 32-draw
matches_per_round = {
    1: 16,  # Round of 32 (16 matches)
    2: 8,   # Round of 16 (8 matches)
    3: 4,   # Quarterfinals (4 matches)
    4: 2,   # Semifinals (2 matches)
    5: 1    # Grand Final (1 match)
}

# 3. 🟢 ADAPTATION: Update round labels for a 32-draw
round_labels = {
    1: "Round of 32",
    2: "Round of 16",
    3: "Quarterfinals",
    4: "Semifinals",
    5: "Grand Final"
}

# Define flag replacement mappings here (you can expand this easily)
flag_replacements = {
    "cro": "hr",
    "ger": "de",
    "slo": "si",
    "uru": "uy",
    "chi": "cl",
    "gre": "gr",
    "sui": "ch",
    "den": "dk",
    "bul": "bg",
    "por": "pt",
    "tpe": "tw",
    "ned": "nl",
    "rsa": "za",
    "mon": "mc",
    "par": "py",
    "lat": "lv",
    "zim": "zw",
    "phi": "ph",
    "crc": "cr",
    "bar": "bb",
    # Add more as needed
    # "xxx": "yy",
}

# Convert matches into a list of processed matches (if any)
processed_matches = []

for match_index, match in enumerate(matches, start=1):
    players = [p.text.strip().split("(")[0].strip() for p in match.select(".name a")]
    
    # 🏳️ Extract flag codes (if present)
    flag_elements = match.select(".country use")
    flags = []
    for f in flag_elements:
        href = f.get("href", "")
        flag_code = ""
        if "#flag-" in href:
            flag_code = href.split("#flag-")[1].lower()

            # 🔄 Replace flag code if in replacement dictionary
            if flag_code in flag_replacements:
                flag_code = flag_replacements[flag_code]
        flags.append(flag_code)

    score_blocks = match.select(".scores")
    match_data = {"players": players, "flags": flags, "scores": []}

    if len(score_blocks) == 2:
        scores_p1 = [s.text.strip() for s in score_blocks[0].select(".score-item span:first-child") if s.text.strip()]
        scores_p2 = [s.text.strip() for s in score_blocks[1].select(".score-item span:first-child") if s.text.strip()]
        match_data["scores"] = list(zip(scores_p1, scores_p2))

    processed_matches.append(match_data)

# Generate bracket output round by round
match_counter = 0
for round_num, num_matches in matches_per_round.items():
    wiki_output.append(f"\n")
    for match_in_round in range(1, num_matches + 1):
        round_match_id = f"R{round_num}M{match_in_round}"

        if match_counter < len(processed_matches):
            m = processed_matches[match_counter]
            match_counter += 1

            p1 = m["players"][0] if len(m["players"]) > 0 else ""
            p2 = m["players"][1] if len(m["players"]) > 1 else ""

            # 🏁 Get flags (default empty if missing)
            flag1 = m["flags"][0] if len(m["flags"]) > 0 else ""
            flag2 = m["flags"][1] if len(m["flags"]) > 1 else ""

            # Change |bestof=5 to |bestof=3, and change the range for maps from 5 to 3.
            match_entry = f"""|{round_match_id}={{{{Match
    |bestof=3
    |date=
    |opponent1={{{{SoloOpponent|{p1}|flag={flag1}}}}}
    |opponent2={{{{SoloOpponent|{p2}|flag={flag2}}}}}"""

            # Add map data
            # 🟢 ADAPTATION: Iterate up to 3 for best-of-3 sets
            for i in range(3): 
                if i < len(m["scores"]):
                    score1, score2 = m["scores"][i]
                    finished = "true" if score1 or score2 else "skip"
                else:
                    score1 = score2 = ""
                    finished = "skip"
                match_entry += f"""
    |map{i+1}={{{{Map|map=Set {i+1}|score1={score1}|score2={score2}|finished={finished}}}}}"""

            match_entry += """
    }}"""
            wiki_output.append(match_entry)
        else:
            # Empty slot (no match data found)
            wiki_output.append(f"|{round_match_id}=")

wiki_output.append("}}") # Close the bracket

# Output result
display(HTML('''
<button onclick="navigator.clipboard.writeText(this.parentElement.querySelector('pre').innerText)" 
style="background:#4CAF50;color:white;border:none;padding:8px 12px;border-radius:6px;cursor:pointer;">
Copy code
</button>
<pre style="background:#1e1e1e;color:#dcdcdc;padding:12px;border-radius:8px;overflow-x:auto;white-space:pre-wrap;">{}</pre>
'''.format("\n".join(wiki_output))))